In [6]:
%load_ext autoreload
%autoreload 5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from utils.query_utils import athena_query_to_df
import json

In [7]:
games_df_query = """select "date", white, black, "result", 1 as count,
case
	when white = 'luckleland' and result = '1-0' then 1
	when black = 'luckleland' and result = '0-1' then 1
	else 0 
end as "win",
case
	when black = 'luckleland' and result = '1-0' then 1
	when white = 'luckleland' and result = '0-1' then 1
	else 0
end as "loss",
case
	when result = '1/2-1/2' then 1
	else 0
end as "draw",
gamestring, id_key
from lichess.lichess_api_data
order by date desc"""

In [8]:
games_df = athena_query_to_df(games_df_query)

In [9]:
def pgn_parser(game_string):
    last_move = False
    move_number = 1
    game_dict = dict()
    while not last_move:
        move_string = str(move_number) + "."
        next_move_string = str(move_number + 1) + "."
        if next_move_string not in game_string:
            last_move = True
            move_list = game_string[game_string.index(move_string) + len(move_string):].split()
            game_dict.update({move_string[:-1] + "a": move_list[0]})
            if move_list[1] != '0-1' and move_list[1] != '1-0' and move_list[1] != '1/2-1/2':
                game_dict.update({move_string[:-1] + "b": move_list[1]})
        else:
            move_list = game_string[game_string.index(move_string) + len(move_string):game_string.index(next_move_string)].split()  
            game_dict.update({move_string[:-1] + "a": move_list[0]})
            game_dict.update({move_string[:-1] + "b": move_list[1]})
            move_number += 1
    return json.dumps(game_dict)

In [10]:
games_df['game_json'] = games_df['gamestring'].apply(lambda x: pgn_parser(x))

In [11]:
games_df['id_key'] = games_df['id_key'].astype(str)

In [13]:
games_df.drop(['result','gamestring'], axis = 1).to_parquet("s3://jcrasto-chess-analysis/luckleland_results/",partition_cols=["date"])